In [1]:
import os
import numpy as np
import pandas as pd

from quantifier_recommender import QuantifierRecommender

quantifier_recommender = QuantifierRecommender()

# quantifier_recommender.construct_meta_table(datasets_path="./datasets/",
#                                             train_data_path="./data/train_data/",
#                                             test_data_path="./data/test_data/",
#                                             supervised=True)

# quantifier_recommender.evaluation_table.to_csv("./recommender_evaluation_table.csv", index=False)
# quantifier_recommender.meta_features_table.to_csv("./recommender_meta_features_table.csv", index=False)

df = pd.read_csv("./recommender_evaluation_table2.csv")

In [39]:
df = pd.read_csv('./test.csv')

iterables = [df['quantifier'].unique().tolist(), df['dataset'].unique().tolist()]
index = pd.MultiIndex.from_product(iterables, names=["quantifier", "dataset"])

abs_error_list = df["abs_error"].tolist()
run_time_list = df["run_time"].tolist()
new_df = pd.DataFrame({"abs_error": abs_error_list, "run_time": run_time_list}, index=index)
# df.pop('quantifier')
# df.pop('dataset')

# df

new_df.loc['ACC', 'AedesSex']['abs_error']

# X_train = meta_features.values
# y_train = new_df.loc['DyS']['abs_error'].values

# # CC
# fit(X_train, y_train)

# rec_dict = {"CC": rec, "ACC": rec}

# dataset_novo -> extrai meta features -> linha de metafeatures




# y_train
# # # index.get_loc()

np.float64(0.0144999999999999)

In [26]:
import os
import numpy as nps
import pandas as pd
import pdb
from copy import deepcopy
from sklearn.ensemble import RandomForestRegressor
import numpy as np
from sklearn.preprocessing import MinMaxScaler

from quantifier_recommender import QuantifierRecommender

qtf_rec = QuantifierRecommender.load_model("s_qtf_recommender.pkl")
evaluation_table = qtf_rec.evaluation_table

# recommenders = qtf_rec.rec

y_test = None
y_train = None

recommender_evaluation_table = pd.DataFrame(columns=["predicted_error", "true_error"], index=qtf_rec.evaluation_table.index)
for quantifier, recommender in qtf_rec.recommender_dict.items():
    recommender_ = None
    scaler = MinMaxScaler()
    if isinstance(recommender, RandomForestRegressor):
        recommender_ = RandomForestRegressor()

    for dataset in qtf_rec.evaluation_table.index.levels[1]:
        unscaled_X_test = qtf_rec._unscaled_meta_features_table.loc[dataset].values
        y_test = qtf_rec.evaluation_table.loc[quantifier, dataset]['abs_error']

        unscaled_X_train = qtf_rec._unscaled_meta_features_table.drop(index=dataset).values
        y_train = qtf_rec.evaluation_table.loc[quantifier].drop(index=dataset)['abs_error'].values

        scaler.fit(unscaled_X_train)
        X_train = scaler.transform(unscaled_X_train)
        recommender_.fit(X_train, y_train)

        X_test = scaler.fit_transform(np.array(unscaled_X_test).reshape(1, -1))
        predicted_error = recommender_.predict(X_test)[0]

        recommender_evaluation_table.loc[(quantifier, dataset)] = [predicted_error, y_test]

        
        break
    # break
#     for quantifier in self.evaluation_table.index.levels[0].tolist():
#         y_train = self.evaluation_table.loc[quantifier]['abs_error'].values
#         self.recommender_dict[quantifier] = RandomForestRegressor()
#         self.recommender_dict[quantifier].fit(X_train, y_train)
#         print(index)
        # test_data = evaluation_table.loc[index]
        # train_data = evaluation_table.drop(index=index)
        # break



# train_data
# y_train
recommender_evaluation_table
# qtf_rec.evaluation_table.index

predicted_error true_error
quantifier dataset                               
ACC        AedesQuinx          0.01736     0.0465
           AedesSex                NaN        NaN
           anuranCalls             NaN        NaN
CC         AedesQuinx          0.03391      0.092
           AedesSex                NaN        NaN
           anuranCalls             NaN        NaN
DyS        AedesQuinx          0.01114     0.0425
           AedesSex                NaN        NaN
           anuranCalls             NaN        NaN
HDy        AedesQuinx          0.01795     0.0605
           AedesSex                NaN        NaN
           anuranCalls             NaN        NaN
MAX        AedesQuinx          0.01524      0.048
           AedesSex                NaN        NaN
           anuranCalls             NaN        NaN
MS         AedesQuinx          0.01641      0.046
           AedesSex                NaN        NaN
           anuranCalls             NaN        NaN
PACC       AedesQuinx            0.022     0.0795
           AedesSex                NaN        NaN
           anuranCalls             NaN        NaN
PCC        AedesQuinx           0.0365      0.144
           AedesSex                NaN        NaN
           anuranCalls             NaN        NaN
SMM        AedesQuinx          0.01703      0.046
           AedesSex                NaN        NaN
           anuranCalls             NaN        NaN
SORD       AedesQuinx          0.01486      0.042
           AedesSex                NaN        NaN
           anuranCalls             NaN        NaN
X          AedesQuinx         0.018855     0.0475
           AedesSex                NaN        NaN
           anuranCalls             NaN        NaN

In [6]:
qtf_rec.recommender_dict['ACC']

RandomForestRegressor()

In [7]:
from sklearn.ensemble import RandomForestRegressor
isinstance(qtf_rec.recommender_dict['ACC'], RandomForestRegressor)

True

In [53]:
qtf_rec.evaluation_table

abs_error  run_time
quantifier dataset                         
ACC        AedesQuinx      0.0465  0.001329
           AedesSex        0.0145  0.001281
           anuranCalls     0.0200  0.001361
CC         AedesQuinx      0.0920  0.000008
           AedesSex        0.0190  0.000010
           anuranCalls     0.0400  0.000010
DyS        AedesQuinx      0.0425  0.049736
           AedesSex        0.0075  0.048827
           anuranCalls     0.0140  0.048127
HDy        AedesQuinx      0.0605  0.219288
           AedesSex        0.0085  0.217841
           anuranCalls     0.0260  0.208500
MAX        AedesQuinx      0.0480  0.000203
           AedesSex        0.0130  0.000209
           anuranCalls     0.0165  0.000203
MS         AedesQuinx      0.0460  0.030952
           AedesSex        0.0130  0.031222
           anuranCalls     0.0185  0.031122
PACC       AedesQuinx      0.0795  0.004147
           AedesSex        0.0220  0.004062
           anuranCalls     0.0220  0.004088
PCC        AedesQuinx      0.1440  0.002745
           AedesSex        0.0295  0.002721
           anuranCalls     0.0545  0.002634
SMM        AedesQuinx      0.0460  0.000151
           AedesSex        0.0100  0.000149
           anuranCalls     0.0195  0.000115
SORD       AedesQuinx      0.0420  1.379702
           AedesSex        0.0115  1.385671
           anuranCalls     0.0175  0.361445
X          AedesQuinx      0.0475  0.000423
           AedesSex        0.0145  0.000419
           anuranCalls     0.0210  0.000398

In [26]:
y_train = qtf_rec.evaluation_table.loc[quantifier]['abs_error'].values
y_train

array([0.0465, 0.0145, 0.02  ])

In [23]:
qtf_rec.recommender_dict.keys()

dict_keys(['ACC', 'CC', 'DyS', 'HDy', 'MAX', 'MS', 'PACC', 'PCC', 'SMM', 'SORD', 'X'])

In [13]:
import pandas as pd
import numpy as np

old_win_count = 0
win_count = 0
draw_count = 0

old_rec = pd.read_csv("./recommender_data/OLD_recommender_evaluation_table_2.csv")
rec = pd.read_csv("./recommender_data/recommender_evaluation_table_2.csv")

predicted_error = old_rec['predicted_error'].values
true_error = old_rec['true_error'].values
old_rec_error = abs(predicted_error - true_error)

predicted_error = rec['predicted_error'].values
true_error = rec['true_error'].values
rec_error = abs(predicted_error - true_error)

for i in range(len(old_rec_error)):
    if rec_error[i] > old_rec_error[i]:
        win_count += 1
    elif rec_error[i] < old_rec_error[i]:
        old_win_count += 1
    else:
        draw_count += 1


print(win_count)
print(old_win_count)
print(draw_count)

177
43
0


In [1]:
from quantifier_recommender import QuantifierRecommender

qtf_rec = QuantifierRecommender.load_model("s_qtf_recommender.pkl")

qtf_rec.save_meta_table("./recommender_data/meta_features_table.csv", "./recommender_data/evaluation_table.csv")

In [3]:
qtf = QuantifierRecommender()

qtf.load_meta_table("./recommender_data/meta_features_table.csv", "./recommender_data/evaluation_table.csv")

qtf.meta_features_table

,attr_conc.mean,attr_conc.sd,attr_ent.mean,attr_ent.sd,attr_to_inst,best_node.mean,best_node.sd,can_cor.mean,can_cor.sd,cat_to_num,...,tree_imbalance.sd,tree_shape.mean,tree_shape.sd,var.mean,var.sd,var_importance.mean,var_importance.sd,w_lambda,worst_node.mean,worst_node.sd
AedesQuinx,0.0492,0.013558,1.000000,0.000078,0.0,0.000000,0.450229,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.189334
AedesSex,0.0000,0.000000,0.999554,0.000000,0.0,1.000000,0.000000,0.0,0.0,0.0,...,0.658878,0.435942,0.559867,0.0,0.0,0.0,1.000000,0.0,0.176336,1.000000
anuranCalls,1.0000,1.000000,0.000000,1.000000,1.0,0.227146,1.000000,0.0,0.0,0.0,...,1.000000,1.000000,1.000000,0.0,0.0,1.0,0.442822,0.0,1.000000,0.000000


In [5]:
qtf = QuantifierRecommender()
qtf.load_meta_table_and_fit("./recommender_data/meta_features_table.csv", "./recommender_data/evaluation_table.csv")

,attr_conc.mean,attr_conc.sd,attr_ent.mean,attr_ent.sd,attr_to_inst,best_node.mean,best_node.sd,can_cor.mean,can_cor.sd,cat_to_num,...,tree_imbalance.sd,tree_shape.mean,tree_shape.sd,var.mean,var.sd,var_importance.mean,var_importance.sd,w_lambda,worst_node.mean,worst_node.sd
AedesQuinx,0.019865,0.016661,4.723763,0.427495,0.001125,0.742375,0.137430,0.0,0.0,0.0,...,0.050013,0.003325,0.009905,0.0,0.0,0.037037,0.072402,0.0,0.572500,0.076364
AedesSex,0.017272,0.015079,4.723431,0.427459,0.001125,0.950167,0.054558,0.0,0.0,0.0,...,0.076568,0.021071,0.035680,0.0,0.0,0.037037,0.169853,0.0,0.596042,0.112341
anuranCalls,0.069978,0.131726,3.980382,0.889031,0.003341,0.789574,0.238623,0.0,0.0,0.0,...,0.090316,0.044033,0.055943,0.0,0.0,0.045455,0.115556,0.0,0.706004,0.067961
